In [3]:
import numpy as np
from scipy import stats

In [33]:
class hedge:
    def __init__(self, current_pf):
        
        # input in the form [2,'-4p']
        if not isinstance(current_pf, list):
            raise ValueError('Please initialize the current portfolio as a list of two components.')
        if len(current_pf)!=2:
            raise ValueError('Please initialize with (1) how many Delta1 products (2) how many option products.\n Use - or + to indicate short or long.')
            
        self.pf = current_pf
        
        if self.pf[1][0]=='+':
            self.amount = float(self.pf[1][1])
        if self.pf[1][0]=='-':
            self.amount = -float(self.pf[1][1])
            
        if (self.amount==0) and (self.pf[0]==0):
            raise ValueError('No asset in the current portfolio.')
        
        self.style = self.pf[1][2]
        
    def call_delta(self,t,s,K,T,sigma,r):
        return stats.norm.cdf((np.log(s/K)+((r+0.5*sigma**2)*(T-t)))/(sigma*(np.sqrt(T-t))))
    
    def call_gamma(self,t,s,K,T,sigma,r):
        d1 = (np.log(s/K)+((r+0.5*sigma**2)*(T-t)))/(sigma*(np.sqrt(T-t)))
        return stats.norm.pdf(d1)/(s*sigma*np.sqrt(T-t))
        
    def delta_hedging(self,t,s,K,T,sigma,r):
        
        delta = self.call_delta(t,s,K,T,sigma,r)
        
        if (self.pf[0]==0) and (self.style=='c'):
            print('Hedged Portfolio:')
            return -self.amount*delta, 'Delta1', self.amount, 'Call'

        if (self.pf[0]==0) and (self.style=='p'):
            print('Hedged Portfolio:')
            return -self.amount*(delta-1), 'Delta1', self.amount, 'Put'

        if (self.amount==0) and (self.style=='c'):
            print('Hedged Portfolio:')
            return self.pf[0], 'Delta1', -self.pf[0]/delta, 'Call'

        if (self.amount==0) and (self.style=='p'):
            print('Hedged Portfolio:')
            return self.pf[0], 'Delta1', -self.pf[0]/(delta-1), 'Put'

        if (self.amount!=0) and (self.pf[0]!=0):
            if self.style=='c':
                st = 'Call'
            if self.style=='p':
                delta = delta-1
                st = 'Put'

            q1 = -self.amount*delta
            s1 = 'Delta1'

            q2 = -self.pf[0]/delta
            s2 = st
            
            print('Two alternatives are available:')
            return [q1,s1,self.amount,'Call'*(self.style=='c')+'Put'*(self.style=='p')],[self.pf[0],'Delta1',q2,s2]
        
    def gamma_hedging(self,t,s,K,T,sigma,r): 
        
        if self.amount ==0:
            print('Pure Delta=1 portfolio has Gamma=0 yet.')
            return None
        
        gamma = self.call_gamma(t,s,K,T,sigma,r)
        delta = self.call_delta(t,s,K,T,sigma,r)
        
        if (self.pf[0]!=0) and (self.amount!=0): 
            if self.style == 'c':
                
                A = np.zeros((2,2))
                A[0,0] = self.amount * delta
                A[0,1] = delta - 1
                A[1,0] = self.amount * gamma
                A[1,1] = gamma
                b = [-self.pf[0], 0]

                x = np.linalg.solve(A,b)
                print('Hedged Porfolio:')
                return self.pf[0], 'Delta1', x[0],'Call',x[1],'Put'
            
            if self.style == 'p':

                A = np.zeros((2,2))
                A[0,0] = self.amount * (delta-1)
                A[0,1] = delta 
                A[1,0] = self.amount * gamma
                A[1,1] = gamma
                b = [-self.pf[0], 0]

                x = np.linalg.solve(A,b)

                return self.pf[0], 'Delta1', x[0],'Put',x[1],'Call'
        
        if self.pf[0]==0:
            if self.style == 'c':
                return -1, 'Delta1', self.amount, 'Call', -self.amount, 'Put'
            if self.style == 'p':
                return 1, 'Delta1',self.amount, 'Put', -self.amount, 'Call'
        

In [36]:
h = hedge([1,'+2p'])
h.gamma_hedging(0,110,120,1,0.3,0.03)

(1, 'Delta1', 0.5, 'Put', -1.0, 'Call')